# Compress and Compare Image Examples
Example usage to run compression experiments for CIFAR-10, CelebA, and ImageNet models.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import models
import resnet
import compression

from compression_performance import analyze_performance
from experiments import run_compression_experiment

In [3]:
OUTPUT_DIR = './results/usage_examples'
CIFAR_DIR = os.path.join(OUTPUT_DIR, 'cifar')
CELEBA_DIR = os.path.join(OUTPUT_DIR, 'celeba')
IMAGENET_DIR = os.path.join(OUTPUT_DIR, 'imagenet')

In [4]:
for dir in [OUTPUT_DIR, CIFAR_DIR, CELEBA_DIR, IMAGENET_DIR]:
    os.makedirs(OUTPUT_DIR, exist_ok=True)

## Training and Compressing
Examples in Compress and Compare contain multiple compression experiments. Each compression experiment is run via ``experiments.run_compression_experiment``. Example experiments are shown below.

#### CIFAR-10 ResNet20 model with 50% magnitude pruning, recalibration, and retraining

In [ ]:
cifar_gradientpruned_model = run_compression_experiment(
    dataset_name='cifar10', 
    model_fn=resnet.resnet20, 
    output_dir=CIFAR_DIR,
    checkpoint_name=None, 
    compression_fns=[compression.gradientpruning],
    compression_amounts=[0.5],
    train_num_epochs=1, # 1 used as example; use 200 in experiments
    retrain_num_epochs=[1], # 1 used as example; use 50 in experiments
    recalibrate=True,
    device='cuda',
    model_fn_kwargs={'num_classes': 10, 'quantized': False},
    verbose=True, 
    iterative_compression=False,
    allow_weight_updates=False
)

#### CelebA ResNet18 with iterative 95% magnitude pruning

In [ ]:
celeba_iterativemagnitudepruned_model = run_compression_experiment(
    dataset_name='celeba', 
    model_fn=models.load_model, 
    output_dir=CELEBA_DIR,
    checkpoint_name=None, 
    compression_fns=[compression.magnitudepruning, compression.magnitudepruning],
    compression_amounts=[0.95, 0.95],
    train_num_epochs=1,
    retrain_num_epochs=[1, 1], # 2 used as example; use 15 in experiments
    recalibrate=False,
    device='cuda',
    model_fn_kwargs={'name': 'resnet18', 'pretrained': False},
    verbose=True, 
    iterative_compression=True,
    allow_weight_updates=True
)

#### ImageNet MobileNet V2 model with 8-bit quantization

In [ ]:
imagenet_int8quantized_model = run_compression_experiment(
    dataset_name='imagenet', 
    model_fn=models.load_model, 
    output_dir=IMAGENET_DIR,
    checkpoint_name=None, 
    compression_fns=[compression.quantize],
    compression_amounts=['int8'],
    train_num_epochs=0,
    retrain_num_epochs=[],
    recalibrate=False,
    device='cpu',
    model_fn_kwargs={'name': 'mobilenet_v2', 'pretrained': True},
    verbose=True, 
    iterative_compression=False,
    allow_weight_updates=False
)

## Computing Performance Metrics
Compress and Compare relies on performance metrics and model outputs computed for each model. Metrics are computed using ``compression_performance.analyze_performance``. Example usage is shown below

In [ ]:
analyze_performance(CIFAR_DIR, CIFAR_DIR, 'cifar10');